In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [5]:
# Configurations
input_directory = r"C:\Users\33658\Documents\Celebrity Faces Cropped"  # Dataset cropped
image_size = (224, 224)  # Taille des images pour MobileNetV2
learning_rate = 0.0008
batch_size = 32


In [6]:

# Étape 1 : Préparer les données
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normaliser les pixels
    validation_split=0.2,  # 20% pour la validation
    horizontal_flip=True,  # Augmenter les données
    zoom_range=0.2
)

train_generator = datagen.flow_from_directory(
    input_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

validation_generator = datagen.flow_from_directory(
    input_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 565 images belonging to 8 classes.
Found 138 images belonging to 8 classes.


In [7]:
# Étape 2 : Construire le modèle
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Geler les couches du modèle de base
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Étape 3 : Compiler le modèle
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,984 (9.24 MB)

 Trainable params: 165,000 (644.53 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping

# Configurer Early Stopping
early_stopping = EarlyStopping(
    monitor="val_loss",  # Surveiller la perte sur l'ensemble de validation
    patience=25,          # Nombre d'époques sans amélioration avant d'arrêter
    restore_best_weights=True,  # Restaurer les poids correspondant à la meilleure époque
    verbose=1            # Afficher un message lorsque l'entraînement s'arrête
)


In [38]:
# Étape 4 : Entraîner le modèle
num_epochs = 125

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping]  # Ajouter le callback Early Stopping
)

import matplotlib.pyplot as plt

# Courbe de perte
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Courbe de précision (si surveillée)
if 'accuracy' in history.history:
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    plt.show()


Epoch 1/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 678ms/step - accuracy: 0.2067 - loss: 2.1774 - val_accuracy: 0.5000 - val_loss: 1.5779
Epoch 2/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4375 - loss: 1.5982 - val_accuracy: 0.5556 - val_loss: 1.4610
Epoch 3/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 378ms/step - accuracy: 0.5724 - loss: 1.4134 - val_accuracy: 0.7344 - val_loss: 1.0362
Epoch 4/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5625 - loss: 1.3061 - val_accuracy: 0.7778 - val_loss: 1.1163
Epoch 5/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 409ms/step - accuracy: 0.6409 - loss: 1.1669 - val_accuracy: 0.7188 - val_loss: 0.9136
Epoch 6/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5938 - loss: 1.1647 - val_accuracy: 0.7222 - val_loss: 0.9023
Epoch 7/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 414ms/step - accuracy: 0.7347 - loss: 0.8899 - val_accuracy: 0.8125 - val_loss: 0.7954
Epoch 8/125
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6250 - loss: 0.9726 - val_accuracy: 

In [1]:
# Étape 5 : Sauvegarder le modèle
model.summary()
model.save("actor_recognition_model_update.h5")
model.save('actor_recognition_model_update.keras')
print("Modèle sauvegardé avec succès !")

NameError: name 'model' is not defined

Test du modele

In [29]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

# Charger le modèle
model = load_model("actor_recognition_model_night.h5")

# Charger une image de test
img_path = "/content/drive/MyDrive/Colab Notebooks/Data/cruise_test2.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Prédiction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
class_labels = list(train_generator.class_indices.keys())
print(f"L'acteur prédit est : {class_labels[predicted_class]}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\33658\\PycharmProjects\\Act-O-Matic\\Celebrity Faces Cropped\\Celebrity Faces Cropped\\Brad Pitt002_cc1b9701.jpg'

In [47]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Charger le modèle
model = load_model("actor_recognition_model_update.h5")

# Recompiler le modèle (si nécessaire)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Configurer les chemins
train_directory = r"C:\Users\33658\Documents\Celebrity Faces Cropped"  # Dossier utilisé pour l'entraînement
test_directory = r"C:\Users\33658\Documents\Celebrity Faces Cropped\Johnny Depp"  # Dossier de test
image_size = (224, 224)  # Taille d'entrée des images

# Récupérer les noms des classes depuis les sous-dossiers
datagen = ImageDataGenerator(rescale=1.0 / 255)
generator = datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=32,
    class_mode="categorical"
)
class_labels = list(generator.class_indices.keys())  # Liste des noms de classes
print(f"Classes détectées : {class_labels}")

# Fonction pour prédire une image
def predict_image(image_path, model):
    # Charger et prétraiter l'image
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Faire une prédiction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = predictions[0][predicted_class]

    return predicted_class, confidence

# Évaluer les performances globales
correct_predictions = 0
total_images = 0

# Parcourir les images du dossier et faire des prédictions
for img_name in os.listdir(test_directory):
    img_path = os.path.join(test_directory, img_name)

    # Vérifier que le fichier est une image
    if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
        predicted_class, confidence = predict_image(img_path, model)
        true_class = class_labels.index("Johnny Depp")  # Remplacer par la classe correcte pour ce dossier

        if predicted_class == true_class:
            correct_predictions += 1
        total_images += 1

        print(f"Image: {img_name} => Acteur prédit: {class_labels[predicted_class]} (Confiance: {confidence:.2f})")
    else:
        print(f"Fichier ignoré : {img_name}")

# Calculer et afficher la performance globale
accuracy = (correct_predictions / total_images) * 100 if total_images > 0 else 0
print(f"Total d'images évaluées : {total_images}")
print(f"Prédictions correctes : {correct_predictions}")
print(f"Précision globale : {accuracy:.2f}%")


Found 747 images belonging to 9 classes.
Classes détectées : ['Brad Pitt', 'Hugh Jackman', 'Johnny Depp', 'Leonardo DiCaprio', 'Natalie Portman', 'Robert Downey Jr', 'Tom Cruise', 'Tom Hanks', 'Will Smith']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 786ms/step
Image: 001_2288a4f6.jpg => Acteur prédit: Johnny Depp (Confiance: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Image: 002_331d0423.jpg => Acteur prédit: Johnny Depp (Confiance: 0.66)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 003_64926b97.jpg => Acteur prédit: Johnny Depp (Confiance: 0.92)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 005_9406f32d.jpg => Acteur prédit: Johnny Depp (Confiance: 0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Image: 006_8fc31fd7.jpg => Acteur prédit: Johnny Depp (Confiance: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 007_1bc0bcd6.jpg => Acteur prédit: Johnny Depp (Confiance: 0.79)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Image: 008_35d1be70.jpg => Acteur prédit: Johnny Depp (Confiance: 0.96)
1/1 ━━━━━━━━━━━━━━━━━━